# Modeling the potential of CRISPR gene drives for mosquito suppression
<hr style="border:2px solid gray"> </hr>
This notebook is for generating models with GP utilizing code generated by Sam E Champer.
In 2024/12/20 weitang Sun get this code from Yuan Hu Allegrettie, he use this code to train a machine learning model to predict the outcome of slim simulations. 
First, some imports:

In [14]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MaxAbsScaler

In [2]:
from matplotlib import pyplot as plt

In [3]:
#pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB 660.6 kB/s eta 0:00:03
   - -------------------------------------- 0.1/1.4 MB 550.5 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.4 MB 731.4 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.4 MB 985.7 kB/s eta 0:00:02
   -------- ------------------------------- 0.3/1.4 MB 1.3 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.4 MB 1.6 MB/s eta 0:00:01
   ------------------ --------------------- 0.7/1.4 MB 2.0 MB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.4 MB 2.5 MB/s eta 0:00:01
   -------------------------------------- - 1.4/1.4 MB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 3.3 MB/s eta 0:00:00
Note: you may ne

<hr style="border:1px solid gray"> </hr>

## 0. Data preprocessing

In [3]:
#load data
Train = pd.read_csv("Training_data.csv")
Train.columns = ["suppressed","malaria_eliminated","num_fertile_females","adult_total_numbers","adult_female","wt","dr","r1","r2"
                 ,"carrier_mos_frequency","healthy_mosquito","carrier_mosquito","carrier_mosquito2","patient_mosquito","healthy_human"
                 ,"carrier_human","patient_human","resistant_human","chasing","chasing_start","chasing_end","avg_pop_during_chase",
                 "fertile_female_average","simulation_end",
                 "transmission rate to mosquitoes", 
                 "transmission rate to humans", 
                 "average dispersal",
                 "weekly remating chance",
                 "mosquito reproduction chance", 
                 "animal biting rate",
                 "viability fitness", 
                 "female fecundity fitness",
                 "germline resistance rate",
                 "drive conversion efficiency",
                 "embryo resistance rate", 
                 "functional resistance proportion",
                 "low-density growth rate",
                 "seasonal population change",
                 "infection duration",
                 "immunity duration",
                 "density_num_adult_female","human_density","sim_bound"]

Test = pd.read_csv("230717_testdata.csv")
Test.columns = ["suppressed","malaria_eliminated","num_fertile_females","adult_total_numbers","adult_female","wt","dr","r1","r2",
                "carrier_mos_frequency","healthy_mosquito","carrier_mosquito","carrier_mosquito2","patient_mosquito","healthy_human",
                "carrier_human","patient_human","resistant_human","chasing","chasing_start","chasing_end","avg_pop_during_chase",
                "fertile_female_average","simulation_end",
                "transmission rate to mosquitoes", 
                 "transmission rate to humans", 
                 "average dispersal",
                 "weekly remating chance",
                 "mosquito reproduction chance", 
                 "animal biting rate",
                 "viability fitness", 
                 "female fecundity fitness",
                 "germline resistance rate",
                 "drive conversion efficiency",
                 "embryo resistance rate", 
                 "functional resistance proportion",
                 "low-density growth rate",
                 "seasonal population change",
                 "infection duration",
                 "immunity duration",
                "density_num_adult_female",
                "human_density","sim_bound"]

newname=['transmission rate to mosquitoes', 
        'transmission rate to humans', 
        'average dispersal',
        'weekly remating chance',
        'mosquito reproduction chance', 
        'animal biting rate',
        'viability fitness', 
        'female fecundity fitness',
        'germline resistance rate',
        'drive conversion efficiency',
        'embryo resistance rate', 
        'functional resistance proportion',
        'low-density growth rate',
        'seasonal population change',
        'infection duration',
        'immunity duration']
#show training set example
#Train['fertile_female_average'].describe()
#Train

C:\Users\19628\AppData\Local\Temp\ipykernel_38100\1709845985.py:2: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  Train = pd.read_csv("Training_data.csv")
C:\Users\19628\AppData\Local\Temp\ipykernel_38100\1709845985.py:25: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  Test = pd.read_csv("230717_testdata.csv")


In [4]:
mosquito_parameter=['average dispersal','weekly remating chance','mosquito reproduction chance','animal biting rate','viability fitness','female fecundity fitness','germline resistance rate','drive conversion efficiency',
                             'embryo resistance rate','functional resistance proportion','low-density growth rate','seasonal population change']

In [5]:
Train_ave=pd.read_csv('Train_ave2.csv')
Test_ave=pd.read_csv('Test_ave2.csv')

In [15]:

#normalize data
abs_scaler = MaxAbsScaler()

# calculate the maximum absolute value for scaling the data using the fit method
abs_scaler.fit(Train_ave.loc[:,newname])
# the maximum absolute values calculated by the fit method
abs_scaler.max_abs_
# transform the data using the parameters calculated by the fit method (the maximum absolute values)
scaled_data = abs_scaler.transform(Train_ave.loc[:,newname])
# store the results in a data frame
Train_scaled = pd.DataFrame(scaled_data, columns=newname)
Train_scaled.describe()


transmission rate to mosquitoes  transmission rate to humans  \
count                      9775.000000                  9775.000000   
mean                          0.529061                     0.550859   
std                           0.233420                     0.218967   
min                           0.185185                     0.222222   
25%                           0.322593                     0.360000   
50%                           0.495185                     0.521111   
75%                           0.720370                     0.727778   
max                           1.000000                     1.000000   

       average dispersal  weekly remating chance  \
count        9775.000000             9775.000000   
mean            0.660457                0.503507   
std             0.211884                0.289434   
min             0.200000                0.000000   
25%             0.518788                0.254500   
50%             0.675152                0.503000   
75%             0.840000                0.756000   
max             1.000000                1.000000   

       mosquito reproduction chance  animal biting rate  viability fitness  \
count                   9775.000000         9775.000000        9775.000000   
mean                       0.504852            0.503797           0.913147   
std                        0.182296            0.290514           0.052600   
min                        0.250000            0.000000           0.800000   
25%                        0.358000            0.251000           0.872000   
50%                        0.463000            0.508000           0.915000   
75%                        0.618000            0.753000           0.958000   
max                        1.000000            1.000000           1.000000   

       female fecundity fitness  germline resistance rate  \
count               9775.000000               9775.000000   
mean                   0.777835                  0.502165   
std                    0.131827                  0.287683   
min                    0.500000                  0.000000   
25%                    0.671000                  0.256707   
50%                    0.780000                  0.504404   
75%                    0.890000                  0.747798   
max                    1.000000                  1.000000   

       drive conversion efficiency  embryo resistance rate  \
count                  9775.000000             9775.000000   
mean                      0.925103                0.375846   
std                       0.051721                0.260670   
min                       0.800000                0.000000   
25%                       0.900000                0.166917   
50%                       0.934000                0.334834   
75%                       0.966000                0.499750   
max                       1.000000                1.000000   

       functional resistance proportion  low-density growth rate  \
count                       9775.000000              9775.000000   
mean                           0.498139                 0.553930   
std                            0.287977                 0.258278   
min                            0.000000                 0.111111   
25%                            0.252000                 0.333333   
50%                            0.494000                 0.555556   
75%                            0.748000                 0.777778   
max                            1.000000                 1.000000   

       seasonal population change  infection duration  immunity duration  
count                 9775.000000         9775.000000        9775.000000  
mean                     0.514501            0.582213           0.572711  
std                      0.292964            0.184769           0.280849  
min                      0.000000            0.333333           0.000000  
25%                      0.250000            0.433333           0.350000  
50%                    

In [16]:

# transform the data using the parameters calculated by the fit method (the maximum absolute values)
scaled_data = abs_scaler.transform(Test_ave.loc[:,newname])
# store the results in a data frame
Test_scaled = pd.DataFrame(scaled_data, columns=newname)
Test_scaled.describe()

transmission rate to mosquitoes  transmission rate to humans  \
count                      9776.000000                  9776.000000   
mean                          0.529254                     0.556754   
std                           0.232166                     0.221148   
min                           0.185185                     0.222222   
25%                           0.325185                     0.363333   
50%                           0.496852                     0.531111   
75%                           0.717407                     0.738889   
max                           0.999630                     1.000000   

       average dispersal  weekly remating chance  \
count        9776.000000             9776.000000   
mean            0.661638                0.504688   
std             0.212128                0.290130   
min             0.200606                0.000000   
25%             0.516970                0.249000   
50%             0.678788                0.505000   
75%             0.838788                0.759000   
max             1.000000                1.000000   

       mosquito reproduction chance  animal biting rate  viability fitness  \
count                   9776.000000         9776.000000        9776.000000   
mean                       0.497770            0.504880           0.913676   
std                        0.178769            0.288148           0.052212   
min                        0.250000            0.000000           0.800000   
25%                        0.355000            0.254000           0.873000   
50%                        0.459000            0.506000           0.916000   
75%                        0.608000            0.756000           0.958000   
max                        1.000000            1.000000           1.000000   

       female fecundity fitness  germline resistance rate  \
count               9776.000000               9776.000000   
mean                   0.775803                  0.497462   
std                    0.133335                  0.289491   
min                    0.500000                  0.000100   
25%                    0.668000                  0.245746   
50%                    0.780000                  0.496146   
75%                    0.888250                  0.749850   
max                    1.000000                  0.999900   

       drive conversion efficiency  embryo resistance rate  \
count                  9776.000000             9776.000000   
mean                      0.925476                0.369107   
std                       0.052253                0.258795   
min                       0.800000                0.000125   
25%                       0.900000                0.161726   
50%                       0.934000                0.327330   
75%                       0.967000                0.495310   
max                       1.000000                1.000375   

       functional resistance proportion  low-density growth rate  \
count                       9776.000000              9776.000000   
mean                           0.500700                 0.552873   
std                            0.287841                 0.257879   
min                            0.000000                 0.111111   
25%                            0.251000                 0.333333   
50%                            0.500000                 0.555556   
75%                            0.750000                 0.777778   
max                            1.000000                 1.000000   

       seasonal population change  infection duration  immunity duration  
count                 9776.000000         9776.000000        9776.000000  
mean                     0.512607            0.585737           0.571087  
std                      0.294223            0.187079           0.283783  
min                      0.000000            0.333333           0.000000  
25%                      0.250000            0.433333           0.350000  
50%                    

In [56]:
X_train = Train_scaled.loc[:,newname]
y_train = Train_ave.loc[:,['malaria_eliminated', 'malaria_prevalence','suppressed','mosquito_prev']]
X_test = Test_scaled.loc[:,newname]
y_test = Test_ave.loc[:,['malaria_eliminated','malaria_prevalence', 'suppressed','mosquito_prev']]
y_test

malaria_eliminated  malaria_prevalence  suppressed  mosquito_prev
0                    0.0            0.235375         0.0       1.000000
1                    0.0            0.537146         0.0       1.000000
2                    0.0            0.505437         0.0       1.000000
3                    0.0            0.567104         0.0       1.000000
4                    0.1            0.040708         0.0       0.709318
...                  ...                 ...         ...            ...
9771                 0.0            0.424042         0.0       1.000000
9772                 0.0            0.390562         0.0       1.000000
9773                 0.0            0.465958         0.0       1.000000
9774                 0.0            0.373896         0.0       1.000000
9775                 0.0            0.458875         0.0       1.000000

[9776 rows x 4 columns]

In [19]:
def check_accuracy(x,y):
    y=y.clip(0,1)
    """
        Check errors between predicted values and known result values.
        x: target value
        y: predicted value
    """
    rmse = np.sqrt(sum((x - y)**2 / len(x)))
    mae =sum(np.abs(x - y) / len(x))
    actual_success = 0
    true_positives = 0
    false_negatives = 0
    false_positives = 0
    for i in range(len(x)):
        if x[i] >= 0.5:
            actual_success += 1
        if y[i] >= 0.5 and x[i] >= 0.5:
            true_positives += 1
        if y[i] < 0.5 and x[i] >= 0.5:
            false_negatives += 1
        if y[i] >= 0.5 and x[i] < 0.5:
            false_positives += 1
    precision = 0
    recall = 0
    f1 = 0
    if true_positives + false_positives:
        precision = true_positives / (true_positives + false_positives)
    if actual_success:
        recall = true_positives / actual_success
    if precision + recall:
        f1 = 2 * precision *recall / (precision + recall)
    return rmse, mae,precision, recall, f1

In [36]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

def train_lgb_model(feature, parameters):
    #  LightGBM dataset
    train_data = lgb.Dataset(X_train[parameters], label=y_train[feature])
    test_data = lgb.Dataset(X_test[parameters], label=y_test[feature])
    
    # parameters
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.2,
        'feature_fraction': 0.9,
    }
    
    # train the model
    num_round = 100
    bst = lgb.LGBMRegressor(**params, n_estimators=num_round)
    bst.fit(X_train[parameters], y_train[feature])
    
    # prediction
    y_pred = bst.predict(X_test[parameters])

    # validation
    mse = mean_squared_error(y_test[feature], y_pred)
    print(f'Root Mean Squared Error: {mse**0.5}')
    

    rmse, mae, precision, recall, f1 = check_accuracy(y_test[feature], y_pred)
    print(f'f1 score: {f1}')
    
    # save the model
    bst.booster_.save_model(f'models//lgb//lgbmodel20241225_{feature}.txt')

In [43]:
train_lgb_model('suppressed',mosquito_parameter)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000215 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2462
[LightGBM] [Info] Number of data points in the train set: 9775, number of used features: 12
[LightGBM] [Info] Start training from score 0.018517
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Root Mean Squared Error: 0.056069670687366395
f1 score: 0.46009389671361506


In [44]:
train_lgb_model('mosquito_prev',mosquito_parameter)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000211 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2462
[LightGBM] [Info] Number of data points in the train set: 9775, number of used features: 12
[LightGBM] [Info] Start training from score 0.948938
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Root Mean Squared Error: 0.06423928319489317
f1 score: 0.9912979273245083


In [45]:
#['malaria_eliminated','malaria_prevalence', 'suppressed','mosquito_prev']
train_lgb_model('malaria_eliminated',newname)


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000270 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3015
[LightGBM] [Info] Number of data points in the train set: 9775, number of used features: 16
[LightGBM] [Info] Start training from score 0.152351
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Root Mean Squared Error: 0.11267634730573502
f1 score: 0.8027788498649171


In [46]:
train_lgb_model('malaria_prevalence',newname)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000282 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3015
[LightGBM] [Info] Number of data points in the train set: 9775, number of used features: 16
[LightGBM] [Info] Start training from score 0.312343
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Root Mean Squared Error: 0.0628037467701393
f1 score: 0.7176391683433937


<hr style="border:1px solid gray"> </hr>

## 2. Training a XGBoost model




In [47]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [76]:
#pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 445.2 kB/s eta 0:04:41
   ---------------------------------------- 0.1/124.9 MB 469.7 kB/s eta 0:04:26
   ---------------------------------------- 0.1/124.9 MB 722.1 kB/s eta 0:02:53
   ---------------------------------------- 0.2/124.9 MB 958.4 kB/s eta 0:02:11
   ---------------------------------------- 0.3/124.9 MB 1.1 MB/s eta 0:01:49
   ---------------------------------------- 0.5/124.9 MB 1.5 MB/s eta 0:01:22
   ---------------------------------------- 0.6/124.9 MB 1.8 MB/s eta 0:01:11
   ---------------------------------------- 1.0/124.9 MB 2.5 MB/s eta 0:00:49
   ---------------------------------------- 1.3/124.9 MB 2.9 MB/s eta 0:00:43
    --------------------------------------- 1.7/124.9 MB 3.4 MB/s eta 

In [73]:
# 创建 DMatrix
def train_XGBoost_model(feature, parameters):

    dtrain = xgb.DMatrix(X_train[parameters], label=y_train[feature])
    dtest = xgb.DMatrix(X_test[parameters], label=y_test[feature])
    params = {
    'objective': 'reg:squarederror',  # regression
    'max_depth': 5,                    # depth
    'learning_rate': 0.2,             # learning rate
    'colsample_bytree': 0.8,           # feature fraction each tree

    }
    # traning
    model = xgb.train(params, dtrain, num_boost_round=100)
    # prediction
    y_pred = model.predict(dtest)

    # validation
    rmse = mean_squared_error(y_test[feature], y_pred)**0.5
    print(f'Mean Squared Error: {rmse:.2f}')
    rmse, mae,precision, recall, f1=check_accuracy(y_test[feature], y_pred)
    print(f'f1 score: {f1}')
    model.save_model('models//xgb//20241225xgb_model'+feature+'.json')

In [74]:
#['malaria_eliminated','malaria_prevalence', 'suppressed','mosquito_prev']
train_XGBoost_model('suppressed',mosquito_parameter)

Mean Squared Error: 0.06
f1 score: 0.3681592039800995


In [75]:
train_XGBoost_model('mosquito_prev',mosquito_parameter)

Mean Squared Error: 0.07
f1 score: 0.9918810628426824


In [76]:
train_XGBoost_model('malaria_eliminated',newname)

Mean Squared Error: 0.12
f1 score: 0.7926013380558835


In [77]:
train_XGBoost_model('malaria_prevalence',newname)

Mean Squared Error: 0.07
f1 score: 0.6502534395365677


<hr style="border:1px solid gray"> </hr>

## 2. Training a CatBoost model




In [98]:
#pip install catboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.8 MB 660.6 kB/s eta 0:02:35
   ---------------------------------------- 0.1/101.8 MB 655.4 kB/s eta 0:02:36
   ---------------------------------------- 0.1/101.8 MB 774.0 kB/s eta 0:02:12
   ---------------------------------------- 0.3/101.8 MB 1.1 MB/s eta 0:01:37
   ---------------------------------------- 0.3/101.8 MB 1.2 MB/s eta 0:01:25
   ---------------------------------------- 0.6/101.8 MB 1.7 MB/s eta 0:01:01
   ---------------------------------------- 0.8/101.8 MB 2.0 MB/s eta 0:00:50
   ---------------------------------------- 1.1/101.8 MB 2.6 MB/s eta 0:00:39
    --------------------------------------- 1.5/101.8 MB 3.2 MB/s eta 0:00:32
    --------------------------------------- 2.0/101.8 MB 3.9 MB/s eta 0:

In [62]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [68]:
def train_CatBoost_model(feature, parameters):

    # initialize model
    model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1, verbose=100,l2_leaf_reg=5)
    # traning
    model.fit(X_train[parameters], y_train[feature])
    # prediction
    predictions = model.predict(X_test[parameters])
    # validation
    rmse, mae,precision, recall, f1=check_accuracy(y_test[feature], predictions)
    print(f'f1 score: {f1}')
    print(f'Mean Squared Error: {rmse}')
    model.save_model('models//catboost//20241225catboost_model'+feature+'.cbm')

In [69]:
#['malaria_eliminated','malaria_prevalence', 'suppressed','mosquito_prev']
train_CatBoost_model('suppressed',mosquito_parameter)

0:	learn: 0.0912371	total: 2.31ms	remaining: 2.31s
100:	learn: 0.0390987	total: 165ms	remaining: 1.47s
200:	learn: 0.0329289	total: 316ms	remaining: 1.26s
300:	learn: 0.0276293	total: 463ms	remaining: 1.07s
400:	learn: 0.0237759	total: 612ms	remaining: 914ms
500:	learn: 0.0211945	total: 751ms	remaining: 748ms
600:	learn: 0.0186288	total: 912ms	remaining: 606ms
700:	learn: 0.0165572	total: 1.07s	remaining: 459ms
800:	learn: 0.0150775	total: 1.24s	remaining: 308ms
900:	learn: 0.0138380	total: 1.4s	remaining: 154ms
999:	learn: 0.0126419	total: 1.57s	remaining: 0us
f1 score: 0.4976958525345621
Mean Squared Error: 0.05135704969515887


In [70]:
train_CatBoost_model('mosquito_prev',mosquito_parameter)

0:	learn: 0.1435750	total: 2.13ms	remaining: 2.13s
100:	learn: 0.0586562	total: 164ms	remaining: 1.46s
200:	learn: 0.0487928	total: 317ms	remaining: 1.26s
300:	learn: 0.0434089	total: 454ms	remaining: 1.05s
400:	learn: 0.0389284	total: 606ms	remaining: 905ms
500:	learn: 0.0349878	total: 749ms	remaining: 746ms
600:	learn: 0.0318892	total: 899ms	remaining: 597ms
700:	learn: 0.0292734	total: 1.04s	remaining: 444ms
800:	learn: 0.0270823	total: 1.2s	remaining: 298ms
900:	learn: 0.0251287	total: 1.34s	remaining: 147ms
999:	learn: 0.0234900	total: 1.49s	remaining: 0us
f1 score: 0.9920413218784588
Mean Squared Error: 0.060246993279887105


In [71]:
train_CatBoost_model('malaria_eliminated',newname)

0:	learn: 0.2642220	total: 2.26ms	remaining: 2.26s
100:	learn: 0.0974719	total: 165ms	remaining: 1.47s
200:	learn: 0.0843821	total: 323ms	remaining: 1.28s
300:	learn: 0.0761613	total: 482ms	remaining: 1.12s
400:	learn: 0.0701644	total: 644ms	remaining: 962ms
500:	learn: 0.0649884	total: 798ms	remaining: 795ms
600:	learn: 0.0604227	total: 948ms	remaining: 629ms
700:	learn: 0.0563420	total: 1.09s	remaining: 465ms
800:	learn: 0.0528547	total: 1.25s	remaining: 310ms
900:	learn: 0.0497848	total: 1.48s	remaining: 163ms
999:	learn: 0.0468943	total: 1.67s	remaining: 0us
f1 score: 0.8107476635514019
Mean Squared Error: 0.10503215707766671


In [72]:
train_CatBoost_model('malaria_prevalence',newname)

0:	learn: 0.1469873	total: 4.61ms	remaining: 4.6s
100:	learn: 0.0727499	total: 175ms	remaining: 1.56s
200:	learn: 0.0565385	total: 343ms	remaining: 1.36s
300:	learn: 0.0475192	total: 509ms	remaining: 1.18s
400:	learn: 0.0416589	total: 672ms	remaining: 1s
500:	learn: 0.0373395	total: 840ms	remaining: 837ms
600:	learn: 0.0337527	total: 1.01s	remaining: 670ms
700:	learn: 0.0310382	total: 1.18s	remaining: 501ms
800:	learn: 0.0286301	total: 1.34s	remaining: 334ms
900:	learn: 0.0265840	total: 1.51s	remaining: 166ms
999:	learn: 0.0248311	total: 1.68s	remaining: 0us
f1 score: 0.8053777208706786
Mean Squared Error: 0.048957042919023745
